python 3.12.2

# tensor基础操作

基础工具

In [ ]:
from PIL import Image # Pillow
import numpy as np
from matplotlib import pyplot as plt

im = Image.open("./jk.png")
im_pillow = np.asarray(im) # 浅拷贝
print(im_pillow.shape)
im_pillow_c1 = im_pillow[:,:,0]
print(im_pillow_c1.shape)
im_pillow_c2 = im_pillow[:,:,1]
im_pillow_c3 = im_pillow[:,:,2]
plt.subplot(2, 2, 1)
plt.imshow(im_pillow)
imzero = np.zeros((im_pillow.shape[0], im_pillow.shape[1], 2))
data = np.concatenate((im_pillow_c1[:,:,np.newaxis], imzero), axis=2)
print(data.dtype)
# plt.imshow(data) 需要元素是uint8类型
plt.subplot(2, 2, 2)
plt.imshow(data.astype(np.uint8))
im_pillow_c2_3ch = np.zeros((im_pillow.shape[0], im_pillow.shape[1], 3))
print(im_pillow_c2_3ch.shape)
im_pillow_c2_3ch[:,:,1] = im_pillow_c2[:,:]
plt.subplot(2, 2, 3)
plt.imshow(im_pillow_c2_3ch.astype(np.uint8))
data = np.concatenate((imzero, im_pillow_c3[:,:,np.newaxis]), axis=2)
plt.subplot(2, 2, 4)
plt.imshow(data.astype(np.uint8))
plt.show()

In [ ]:
import cv2
im_cv2 = cv2.imread('./jk.png')
type(im_cv2)

im_cv2.shape

numpy

In [ ]:
import numpy as np
a = np.arange(6)
print(a.shape)

b = a.view() # 浅拷贝
print(b.shape)
b.shape = 2,3
b[0,0] = 100

print(a)
print(b)

b = np.array(b) # 深拷贝
b[0,0] = 20
print(a, b, sep='\n')

tensor连接操作

In [ ]:
import torch
A = torch.arange(0,4)
B = torch.arange(5,9)
print(A.shape)
C = torch.stack((A,B),0)
print(C)
D = torch.stack((A,B),1)
print(D)
print(torch.stack((A,B),-1)) # 负数倒数维度
print(D == torch.stack((A,B),-1))
print(torch.stack((A,B),-2))

tensor切分操作

In [ ]:
A = torch.tensor(range(1,11))
B = torch.chunk(A, 2, 0)
print(B)
B = torch.chunk(A, 3, 0) # 除法 向上取整
print(B)
A = torch.tensor([1,2,3])
B = torch.chunk(A,5,0)
print(B)

In [ ]:
A = torch.rand(4,4)
B = torch.split(A,2,0)
print(B)
C = torch.split(A,3,0)

A = torch.rand(5,4)
B = torch.split(A,(2,2,1),0) # 为列表时 按数字切分
print(B)

tensor降维切分

In [ ]:
A = torch.arange(0,16).view(4,4)
print(A)
b = torch.unbind(A, 0)
print(b)

tensor索引操作

In [ ]:
A = torch.arange(0,16).view(4,4)
B = torch.index_select(A,0,torch.tensor([1,3]))
print(B)
C = torch.index_select(A,1,torch.tensor([0,3]))
print(C)

In [ ]:
A = torch.rand(6)
C = torch.masked_select(A, A>0.3)
print(C)

数据读取

In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
class MyDataset(Dataset):
    def __init__(self, data_tensor:torch.Tensor, target_tensor:torch.Tensor):
        self.data_tensor = data_tensor
        self.target_tensor = target_tensor
    def __len__(self) -> int:
        return self.data_tensor.size(0)
    def __getitem__(self, index) -> tuple[torch.Tensor, torch.Tensor]:
        return self.data_tensor[index], self.target_tensor[index]

data_tensor = torch.randn(10, 3)
target_tensor = torch.randint(2, (10,))
my_dataset = MyDataset(data_tensor, target_tensor)
print(len(my_dataset))
print(target_tensor, my_dataset[0])
data = DataLoader(dataset=my_dataset, batch_size=2, shuffle=True, num_workers=0)
for data, target in data:
    print(data, target)

---
# torchvision

In [ ]:
import torchvision
mnist_dataset = torchvision.datasets.MNIST(root="../../data", train=True, 
        transform=None, target_transform=None, download=True)
display(mnist_dataset[0][0])
print(mnist_dataset[0][1])

In [ ]:
from PIL import Image
from torchvision import transforms
img = Image.open('jk.png')
display(img)
print(type(img))

imgl = transforms.ToTensor()(img)
print(type(imgl))

img2 = transforms.ToPILImage()(imgl)
print(type(img2))
resize_func = transforms.Resize((200,200), interpolation=2)
new_img = resize_func(img)
display(new_img)

# 剪裁
new_img = transforms.CenterCrop((60,70))(img)
display(new_img)
new_img = transforms.RandomCrop((80,80))(img)
display(new_img)
new_imgs = transforms.FiveCrop((60,70))(img)
for data in new_imgs:
    display(data)

In [ ]:
# 翻转
from PIL import Image
from torchvision import transforms
img = Image.open('jk.png')

new_img = transforms.RandomHorizontalFlip(p=1)(img)
display(new_img)
new_img = transforms.RandomVerticalFlip(p=1)(img)
display(new_img)

In [ ]:
from PIL import Image
from torchvision import transforms

# 标准化
img = Image.open('jk.png')
print(transforms.ToTensor()(img).shape)
# RGBA
data = transforms.ToTensor()(img).index_select(0,torch.tensor([0,1,2]))
print(data.shape)
new_img = transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))(data)
display(transforms.ToPILImage()(new_img))

In [ ]:
from PIL import Image
from torchvision import transforms

# 组合变换
img = Image.open('jk.png')
composed = transforms.Compose([transforms.Resize((200,200)), transforms.RandomCrop(80)])
display(composed(img))

In [ ]:
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
composed = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5),(0.5))])
mnist_dataset = torchvision.datasets.MNIST(root="../../data", train=False, 
        transform=composed, target_transform=None, download=True)
item = mnist_dataset.__getitem__(0)
print(type(item))
print(type(item[0]), item[0].shape)
display(transforms.ToPILImage()(item[0]), item[1])
tensor_dataloader = DataLoader(dataset=mnist_dataset, batch_size=32)
img_tensor, label_tensor = (iter(tensor_dataloader))._next_data()
grid_tensor = torchvision.utils.make_grid(img_tensor, nrow=8, padding=2)
display(transforms.ToPILImage()(grid_tensor))
torchvision.utils.save_image(grid_tensor, "img.jpg")

常见网络模型

In [ ]:
import torch
import torchvision.models as models
googlenet = models.googlenet(pretrained=True)
# fine-tuning
# 分类层输入参数
fc_in_features = googlenet.fc.in_features
print("fc_in_features:", fc_in_features)
# 分类层输入参数
fc_out_features = googlenet.fc.out_features
print("fc_out_features:", fc_out_features)
googlenet.fc = torch.nn.Linear(fc_in_features, 10)